In [1]:
import pandas as pd
import os #usar funcionalidades que são dependentes de sistema operacional

# Definir o caminho para o diretório temporário onde o repositório será clonado
clone_path = '/tmp/Estudos_Pandas'
excel_file_path = os.path.join(clone_path, '1-SEEG10_GERAL-BR_UF_2022.10.27-FINAL-SITE.xlsx')
repository_url = 'https://github.com/YuriArduino/Estudos_Pandas.git'
branch_name = 'data-tests'

# Remover o diretório do repositório se ele já existir para garantir um clone limpo
# Isso evita o erro "destination path '...' already exists and is not an empty directory"
if os.path.exists(clone_path):
    print(f"Removendo diretório existente: {clone_path}")
    !rm -rf {clone_path}

# Clonar o repositório do GitHub, especificando o branch 'data-tests'
print(f"Clonando repositório do branch '{branch_name}' para {clone_path}...")
!git clone --branch {branch_name} {repository_url} {clone_path}

# Mudar para o diretório do repositório clonado
print(f"Mudando para o diretório: {clone_path}")
%cd {clone_path}

# Puxar os arquivos grandes (LFS) do repositório.
# Este passo é crucial pois o arquivo Excel está armazenado usando Git LFS.
print("Puxando arquivos LFS...")
!git lfs pull

# Voltar para o diretório de conteúdo original do Colab
print("Voltando para o diretório original (/content/)...")
%cd /content/

# Verificar se o arquivo Excel existe no caminho esperado após o clone e LFS pull
if os.path.exists(excel_file_path):
    print(f"Arquivo Excel encontrado em: {excel_file_path}")
    # Ler o arquivo Excel para um DataFrame pandas
    # Motor 'openpyxl' para arquivos .xlsx
    print("Lendo o arquivo Excel com pandas...")
    emissoes_gases = pd.read_excel(excel_file_path, engine='openpyxl', sheet_name = 'GEE Estados',)

    # Exibir as primeiras 5 linhas do DataFrame
    print("Exibindo as primeiras linhas do DataFrame:")
    display(emissoes_gases.head())
else:
    print(f"Erro: O arquivo Excel não foi encontrado em {excel_file_path} após o clone e LFS pull.")
    print("Por favor, verifique se o nome do arquivo e o branch estão corretos.")

Removendo diretório existente: /tmp/Estudos_Pandas
Clonando repositório do branch 'data-tests' para /tmp/Estudos_Pandas...
Cloning into '/tmp/Estudos_Pandas'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 106 (delta 43), reused 10 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 4.69 MiB | 5.51 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Mudando para o diretório: /tmp/Estudos_Pandas
/tmp/Estudos_Pandas
Puxando arquivos LFS...
Voltando para o diretório original (/content/)...
/content
Arquivo Excel encontrado em: /tmp/Estudos_Pandas/1-SEEG10_GERAL-BR_UF_2022.10.27-FINAL-SITE.xlsx
Lendo o arquivo Excel com pandas...


/usr/local/lib/python3.12/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


Exibindo as primeiras linhas do DataFrame:


,Nível 1 - Setor,Nível 2,Nível 3,Nível 4,Nível 5,Nível 6,Emissão / Remoção / Bunker,Gás,Estado,Atividade Econômica,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Processos Industriais,Indústria Química,Produção de ABS,NaN,NaN,NaN,Emissão,COVNM (t),SP,OUTRA_IND,...,897.600,897.600,897.600,897.600,897.600,897.600,897.600,897.600,897.600,897.600
1,Processos Industriais,Indústria Química,Produção de Ácido Adípico,NaN,NaN,NaN,Emissão,N2O (t),SP,OUTRA_IND,...,130.000,130.000,130.000,130.000,130.000,130.000,130.000,130.000,130.000,130.000
2,Processos Industriais,Indústria Química,Produção de Ácido Adípico,NaN,NaN,NaN,Emissão,CO (t),SP,OUTRA_IND,...,1380.576,1380.576,1380.576,1380.576,1380.576,1380.576,1380.576,1380.576,1380.576,1380.576
3,Processos Industriais,Indústria Química,Produção de Ácido Adípico,NaN,NaN,NaN,Emissão,NOx (t),SP,OUTRA_IND,...,431.430,431.430,431.430,431.430,431.430,431.430,431.430,431.430,431.430,431.430
4,Processos Industriais,Indústria Química,Produção de Ácido Adípico,NaN,NaN,NaN,Emissão,CO2e (t) GWP-AR2,SP,OUTRA_IND,...,40300.000,40300.000,40300.000,40300.000,40300.000,40300.000,40300.000,40300.000,40300.000,40300.000


In [2]:
emissoes_gases = emissoes_gases[emissoes_gases['Emissão / Remoção / Bunker'] == 'Emissão']
emissoes_gases = emissoes_gases.drop(columns = 'Emissão / Remoção / Bunker')
colunas_emissao = list(emissoes_gases.loc[:,1970:2021].columns)
colunas_info =list(emissoes_gases.loc[:,'Nível 1 - Setor':'Produto'].columns)
emissoes_por_ano = emissoes_gases.melt(
    id_vars = colunas_info, value_vars = colunas_emissao,
    var_name = 'Ano' , value_name = 'Emissão'
    )
emissoes_por_ano.groupby('Gás')
emissao_por_gas = emissoes_por_ano.groupby(
    'Gás')[['Emissão']].sum().sort_values(
        'Emissão', ascending = False
        )

#DataFrame multi-index

##Emissão de gases por setor

In [3]:
gas_por_setor = emissoes_por_ano.groupby(['Gás', 'Nível 1 - Setor'])[['Emissão']].sum()
gas_por_setor

Emissão
Gás      Nível 1 - Setor                                 
C2F6 (t) Processos Industriais               5.765018e+02
CF4 (t)  Processos Industriais               7.135481e+03
CH4 (t)  Agropecuária                        5.928481e+08
         Energia                             3.429034e+07
         Mudança de Uso da Terra e Floresta  8.662166e+07
...                                                   ...
N2O (t)  Resíduos                            2.764127e+05
NOx (t)  Agropecuária                        2.602719e+06
         Energia                             9.361215e+07
         Processos Industriais               3.546360e+06
SF6 (t)  Processos Industriais               4.705800e+02

[72 rows x 1 columns]

`Método cross-section ou xs`

>É importante identificar a ordem dos índices, para identificar em qual level essa informação está localizada.


In [4]:
gas_por_setor.xs('CO2 (t)', level = 0)

,Emissão
Nível 1 - Setor,
Agropecuária,5.411750e+08
Energia,1.261352e+10
Mudança de Uso da Terra e Floresta,3.733930e+10
Processos Industriais,2.729603e+09
Resíduos,2.276264e+07


>A ordem das informações é importante quando usamos o cross-section!

In [5]:
gas_por_setor.xs(('CO2 (t)', 'Mudança de Uso da Terra e Floresta'), level = [0,1])

,,Emissão
Gás,Nível 1 - Setor,
CO2 (t),Mudança de Uso da Terra e Floresta,3.733930e+10


Em uma nova célula, vamos escrever gas_por_setor.xs novamente, mas dessa vez passaremos o parâmetro level = 0 no lugar de 'Mudança de Uso da Terra e Floresta'

In [6]:
gas_por_setor.xs('CO2 (t)', level = 0).max()

,0
Emissão,3.733930e+10


Método combinado
`idxmax()`

In [7]:
gas_por_setor.xs('CO2 (t)', level = 0).idxmax()

,0
Emissão,Mudança de Uso da Terra e Floresta


## 📘 Para saber mais: DataFrames MultiIndex

Um **DataFrame MultiIndex** é uma estrutura de dados em pandas que permite criar um **índice hierárquico**, ou seja, um índice com múltiplos níveis.  
Isso possibilita organizar os dados de maneira mais detalhada do que um índice simples.  

Exemplo de um DataFrame MultiIndex, no qual as duas primeiras colunas representam os 2 níveis do índice:

| Gás       | Nível 1 - Setor                 | Emissão       |
|-----------|---------------------------------|---------------|
| C2F6 (t)  | Processos industriais           | 576           |
| CF4 (t)   | Processos industriais           | 7135          |
| CH4 (t)   | Agropecuária                    | 592848100     |
| CH4 (t)   | Energia                         | 34290340      |
| CH4 (t)   | Mudança de Uso da Terra e Floresta | 86621660   |
| NOx (t)   | Agropecuária                    | 2602719       |
| NOx (t)   | Energia                         | 93612150      |
| NOx (t)   | Processos industriais           | 3546360       |

📌 A hierarquia dos índices é da **esquerda para a direita**:  
- **Level 0** → Gás  
- **Level 1** → Setor  

---

### 🔎 Seleção de dados em MultiIndex

1. **Selecionar diretamente pelo primeiro nível (Level 0):**
```python
dados.loc['CH4 (t)']
````

2. **Selecionar valores passando uma tupla (todos os níveis):**

```python
dados.loc[('CH4 (t)', 'Agropecuária')]
```

3. **Selecionar por um nível específico com `xs()` (cross-section):**

```python
dados.xs('Agropecuária', level=1)
```

---

✅ O `loc` exige respeitar a ordem dos níveis hierárquicos,
enquanto o `xs()` permite "pular" direto para um nível específico.

---


###Emissão máxima

In [8]:
gas_por_setor.groupby(level = 0).idxmax()

,Emissão
Gás,
C2F6 (t),"(C2F6 (t), Processos Industriais)"
CF4 (t),"(CF4 (t), Processos Industriais)"
CH4 (t),"(CH4 (t), Agropecuária)"
CO (t),"(CO (t), Energia)"
CO2 (t),"(CO2 (t), Mudança de Uso da Terra e Floresta)"
CO2e (t) GTP-AR2,"(CO2e (t) GTP-AR2, Mudança de Uso da Terra e F..."
CO2e (t) GTP-AR4,"(CO2e (t) GTP-AR4, Mudança de Uso da Terra e F..."
CO2e (t) GTP-AR5,"(CO2e (t) GTP-AR5, Mudança de Uso da Terra e F..."
CO2e (t) GTP-AR6,"(CO2e (t) GTP-AR6, Mudança de Uso da Terra e F..."


In [9]:
gas_por_setor.groupby(level = 0).max()

,Emissão
Gás,
C2F6 (t),5.765018e+02
CF4 (t),7.135481e+03
CH4 (t),5.928481e+08
CO (t),3.959769e+08
CO2 (t),3.733930e+10
CO2e (t) GTP-AR2,3.850044e+10
CO2e (t) GTP-AR4,3.850044e+10
CO2e (t) GTP-AR5,3.831675e+10
CO2e (t) GTP-AR6,3.843359e+10


In [10]:
valores_max = gas_por_setor.groupby(level = 0).max().values

In [15]:
#insert é capaz de inserir uma nova coluna na tabela
tabela_sumarizada = gas_por_setor.groupby(level = 0).idxmax()
tabela_sumarizada.insert(1, 'Quantidade de emissão',valores_max)
tabela_sumarizada

,Emissão,Quantidade de emissão
Gás,,
C2F6 (t),"(C2F6 (t), Processos Industriais)",5.765018e+02
CF4 (t),"(CF4 (t), Processos Industriais)",7.135481e+03
CH4 (t),"(CH4 (t), Agropecuária)",5.928481e+08
CO (t),"(CO (t), Energia)",3.959769e+08
CO2 (t),"(CO2 (t), Mudança de Uso da Terra e Floresta)",3.733930e+10
CO2e (t) GTP-AR2,"(CO2e (t) GTP-AR2, Mudança de Uso da Terra e F...",3.850044e+10
CO2e (t) GTP-AR4,"(CO2e (t) GTP-AR4, Mudança de Uso da Terra e F...",3.850044e+10
CO2e (t) GTP-AR5,"(CO2e (t) GTP-AR5, Mudança de Uso da Terra e F...",3.831675e+10
CO2e (t) GTP-AR6,"(CO2e (t) GTP-AR6, Mudança de Uso da Terra e F...",3.843359e+10


In [16]:
#swaplevel(), permite inverter um nível de índice com outro
gas_por_setor.swaplevel(0, 1)

Emissão
Nível 1 - Setor                    Gás                   
Processos Industriais              C2F6 (t)  5.765018e+02
                                   CF4 (t)   7.135481e+03
Agropecuária                       CH4 (t)   5.928481e+08
Energia                            CH4 (t)   3.429034e+07
Mudança de Uso da Terra e Floresta CH4 (t)   8.662166e+07
...                                                   ...
Resíduos                           N2O (t)   2.764127e+05
Agropecuária                       NOx (t)   2.602719e+06
Energia                            NOx (t)   9.361215e+07
Processos Industriais              NOx (t)   3.546360e+06
                                   SF6 (t)   4.705800e+02

[72 rows x 1 columns]

In [17]:
#Combinando: swap, groupby, idx & max
gas_por_setor.swaplevel(0, 1).groupby(level = 0).idxmax()

,Emissão
Nível 1 - Setor,
Agropecuária,"(Agropecuária, CO2e (t) GWP-AR5)"
Energia,"(Energia, CO2e (t) GWP-AR6)"
Mudança de Uso da Terra e Floresta,"(Mudança de Uso da Terra e Floresta, CO2e (t) ..."
Processos Industriais,"(Processos Industriais, CO2e (t) GWP-AR6)"
Resíduos,"(Resíduos , CO2e (t) GWP-AR5)"
